In [1]:
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import nltk
from nltk.util import ngrams
import re
nltk.download('punkt')
import pickle
import math
from collections import Counter
import tensorflow as tf
import numpy as np

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [29]:
with open('/home/ec2-user/SageMaker/Data/support_files/result_files.pickle', 'rb') as handle:
    result_files = pickle.load(handle)

In [3]:
index=7
with open(result_files[index], 'r') as f:
        read_f = f.read()
        print([read_f])

['Appeal pending. $$ The plaintiff has appealed from the defendant\'s Order No. I 79-40, dated July 13, 1979. $$ The facts have been stipulated. The plaintiff is a federal savings and loan association, chartered under the Homeowners\' Loan Act of 1933 (12 U.S.C. § 1461 et seq.), administered by the Homeowners\' Loan Corporation. Plaintiff has its principal office in Tacoma, Washington, but also maintains four branch offices within the State of Oregon, located in Portland, Eugene, Springfield and Bend. Each branch is independent of the others, each reporting directly to the principal office. $$ For income tax years before, during and after the income tax year 1974, plaintiff was subject to the Corporation Income Tax Act of 1955 (ORS Chapter 318). A tax is imposed by that act upon the corporate "net income derived from sources within this state, other than income for which the corporation is subject to the tax imposed by the Corporation Excise Tax Law of 1929 (ORS Chapter 317) according 

In [4]:
def get_relevant_path(i):
    with open(result_files[i], 'r') as f:
        read_f = f.read().lower()
        if len(read_f)>3:
            pattern = re.compile(r"\b35 (?:u(?:[. ]+)?s(?:[. ]+)?c(?:[. ]+)?a?(?:[. ]+)?)")
            first_match = pattern.search(read_f)
            if first_match:
                return 1
            else:
                return 0
        else:
            return 0

In [5]:
pool = Pool(processes=cpu_count())
relevant_files=pool.map(get_relevant_path, tqdm(range(len(result_files))))
pool.close()
pool.join()

100%|██████████| 4387241/4387241 [16:27<00:00, 4444.42it/s]   


In [6]:
print(sum(relevant_files))

13055


In [7]:
patent_files=[result_files[i] for i in range(len(relevant_files)) if relevant_files[i]==1]

In [37]:
print(len(patent_files))

13055


In [9]:
index=7
with open(patent_files[index], 'r') as f:
        read_f = f.read()
        print([read_f])

['United States Court of Appeals for the Federal Circuit GENERAL ELECTRIC COMPANY, Appellant, v. INTERNATIONAL TRADE COMMISSION, Appellee, AND MITSUBISHI HEAVY INDUSTRIES, LTD., AND MITSUBISHI POWER SYSTEMS AMERICAS, INC., Intervenors. 2010-1223 Appeal from the United States International Trade Commission in Investigation No. 337-TA-641. Decided: July 6, 2012 WILLIAM F. LEE, Wilmer Cutler Pickering Hale and Dorr LLP, of Boston, Massachusetts, for appellant. With him on the brief were RICHARD W. O\'NEILL, LOUIS W. TOMPROS, and SARAH B. PETTY. GENERAL ELECTRIC v. ITC 2 JAMES A. WORTH, Attorney, Office of General Counsel, United States International Trade Commission, of Washing- ton, DC, for appellee. With him on the brief were JAMES M. LYONS, General Counsel, and WAYNE W. HERRINGTON, Assistant General Counsel $$ DONALD R. DUNNER, Finnegan, Henderson, Farabow, Garrett & Dunner, LLP, of Washington, DC, for intervenors. With him on the brief were THOMAS H. JENKINS, THOMAS W. WINLAND, JEFFRE

In [30]:
non_patent_files=[]
for i in range(0, len(result_files), math.floor(len(result_files)/len(patent_files))):
    with open(result_files[i], 'r') as f:
        read_f = f.read().lower()
        if "patent" not in read_f:
            non_patent_files.append(result_files[i])

In [33]:
print(len(non_patent_files))

12636


In [32]:
with open('/home/ec2-user/SageMaker/for_Brandon/non_patent_files.pickle', 'wb') as handle:
    pickle.dump(non_patent_files, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
index=567
with open(non_patent_files[index], 'r') as f:
        read_f = f.read()
        print([read_f])

['The homicide took place in the public road between Luverne and Brantley, on a Sunday afternoon. According to the testimony of the state\'s witnesses, no one was present at the difficulty except the defendants and the deceased. Several witnesses testified to having seen the difficulty from a distance, but none were close enough to hear what was said by either party. After the difficulty was over and Briley was dead, as a result of pistol wounds at the hands of one or both of the defendants, the defendants left the scene in a Ford car and drove to Brantley, some miles away. After defendants had offered evidence tending to prove self-defense and that, in the fight between Dock Jones and deceased, deceased had Dock Jones down on the ground at the time the fatal shots were fired, and that in the scuffle Jones got sand and dirt in his hair and had his shirt torn, and that his face and shirt were in the same condition when he reached Brantley, the defendant offered to prove by the witness R

In [13]:
non_patent_word_list=[]
for i in range(len(non_patent_files)):
    with open(non_patent_files[i], 'r') as f:
        read_f = f.read()
        wrd_list=read_f.split(" ")
        non_patent_word_list.extend(wrd_list)

In [14]:
print(len(non_patent_word_list))

27923368


In [15]:
print(non_patent_word_list[0:100])

['864', 'F.2d', '201', '$$', '1989', 'A.M.C.', '852,', '12', 'Fed.R.Serv.3d', '1366', '$$', 'Thomas', 'CAREY,', 'et', 'al.,', 'Plaintiffs,', 'Appellants,v.BAHAMA', 'CRUISE', 'LINES,', 'Defendant,', 'Appellee.', '$$', 'No.', '88-1169.', '$$', 'United', 'States', 'Court', 'of', 'Appeals,First', 'Circuit.', '$$', 'Heard', 'Oct.', '5,', '1988.Decided', 'Dec.', '20,', '1988.', '$$', 'Mark', 'J.', 'Albano,', 'with', 'whom', 'Goldman,', 'Albano,', 'Caruso', 'and', 'Fierberg,', 'P.C.', 'was', 'on', 'brief,', 'for', 'appellants.', '$$', 'Richard', 'B.', 'Kydd,', 'with', 'whom', 'Kneeland,', 'Kydd', '&', 'Handy,', 'was', 'on', 'brief,', 'for', 'appellee.', '$$', 'Before', 'BREYER,', 'Circuit', 'Judge,', 'TIMBERS,*', 'Senior', 'Circuit', 'Judge,', 'and', 'TORRUELLA,', 'Circuit', 'Judge.', '$$', 'TORRUELLA,', 'Circuit', 'Judge.', '$$', '1', '$$', 'The', 'plaintiffs,', 'a', 'married', 'couple,', 'are', 'residents', 'of', 'Massachusetts.']


In [16]:
patent_word_list=[]
for i in range(len(patent_files)):
    with open(patent_files[i], 'r') as f:
        read_f = f.read()
        wrd_list=read_f.split(" ")
        patent_word_list.extend(wrd_list)

In [17]:
print(len(patent_word_list))

81828750


In [18]:
print(patent_word_list[0:100])

['United', 'States', 'Court', 'of', 'Appeals', 'for', 'the', 'Federal', 'Circuit', 'CUBIST', 'PHARMACEUTICALS,', 'INC.,', 'Plaintiff-Cross-Appellant', 'v.', 'HOSPIRA,', 'INC.,', 'Defendant-Appellant', '2015-1197,', '2015-1204,', '2015-1259', 'Appeals', 'from', 'the', 'United', 'States', 'District', 'Court', 'for', 'the', 'District', 'of', 'Delaware', 'in', 'No.', '1:12-cv-00367-GMS,', 'Judge', 'Gregory', 'M.', 'Sleet.', 'Decided:', 'November', '12,', '2015', 'WILLIAM', 'F.', 'LEE,', 'Wilmer', 'Cutler', 'Pickering', 'Hale', 'and', 'Dorr', 'LLP,', 'Boston,', 'MA,', 'argued', 'for', 'plaintiff-cross-', 'appellant.', 'Also', 'represented', 'by', 'MARK', 'CHRISTOPHER', 'FLEMING,', 'LISA', 'JON', 'PIROZZOLO;', 'WILLIAM', 'G.', 'MCELWAIN,', 'Washington,', 'DC.', 'JAMES', 'F.', 'HURST,', 'Kirkland', '&', 'Ellis', 'LLP,', 'Chicago,', 'IL,', 'argued', 'for', 'defendant-appellant.', 'Also', 'represented', 'by', 'JOHN', 'C.', "O'QUINN,", 'Washington,', 'DC;', 'LESLIE', 'M.', 'SCHMIDT,', 'New', 'Yo

In [19]:
def extract_ngrams(word_list, n):
    n_grams = ngrams(word_list, n)
    return [ ' '.join(grams) for grams in n_grams]

In [20]:
patent_bigram=extract_ngrams(patent_word_list,2)
patent_trigram=extract_ngrams(patent_word_list,3)
patent_fourgram=extract_ngrams(patent_word_list,4)
patent_fivegram=extract_ngrams(patent_word_list,5)


non_patent_bigram=extract_ngrams(non_patent_word_list,2)
non_patent_trigram=extract_ngrams(non_patent_word_list,3)
non_patent_fourgram=extract_ngrams(non_patent_word_list,4)
non_patent_fivegram=extract_ngrams(non_patent_word_list,5)


In [21]:
print(patent_bigram[0:77])

['United States', 'States Court', 'Court of', 'of Appeals', 'Appeals for', 'for the', 'the Federal', 'Federal Circuit', 'Circuit CUBIST', 'CUBIST PHARMACEUTICALS,', 'PHARMACEUTICALS, INC.,', 'INC., Plaintiff-Cross-Appellant', 'Plaintiff-Cross-Appellant v.', 'v. HOSPIRA,', 'HOSPIRA, INC.,', 'INC., Defendant-Appellant', 'Defendant-Appellant 2015-1197,', '2015-1197, 2015-1204,', '2015-1204, 2015-1259', '2015-1259 Appeals', 'Appeals from', 'from the', 'the United', 'United States', 'States District', 'District Court', 'Court for', 'for the', 'the District', 'District of', 'of Delaware', 'Delaware in', 'in No.', 'No. 1:12-cv-00367-GMS,', '1:12-cv-00367-GMS, Judge', 'Judge Gregory', 'Gregory M.', 'M. Sleet.', 'Sleet. Decided:', 'Decided: November', 'November 12,', '12, 2015', '2015 WILLIAM', 'WILLIAM F.', 'F. LEE,', 'LEE, Wilmer', 'Wilmer Cutler', 'Cutler Pickering', 'Pickering Hale', 'Hale and', 'and Dorr', 'Dorr LLP,', 'LLP, Boston,', 'Boston, MA,', 'MA, argued', 'argued for', 'for plainti

In [22]:
counter_patent_bigram = Counter(patent_bigram)
top_patent_bigram=counter_patent_bigram.most_common(2500)

counter_patent_trigram = Counter(patent_trigram)
top_patent_trigram=counter_patent_trigram.most_common(2500)

counter_patent_fourgram = Counter(patent_fourgram)
top_patent_fourgram=counter_patent_fourgram.most_common(2500)

counter_patent_fivegram = Counter(patent_fivegram)
top_patent_fivegram=counter_patent_fivegram.most_common(2500)

In [23]:
counter_non_patent_bigram = Counter(non_patent_bigram)
top_non_patent_bigram=counter_non_patent_bigram.most_common(10000)

counter_non_patent_trigram = Counter(non_patent_trigram)
top_non_patent_trigram=counter_non_patent_trigram.most_common(10000)

counter_non_patent_fourgram = Counter(non_patent_fourgram)
top_non_patent_fourgram=counter_non_patent_fourgram.most_common(10000)

counter_non_patent_fivegram = Counter(non_patent_fivegram)
top_non_patent_fivegram=counter_non_patent_fivegram.most_common(10000)

In [28]:
print(top_patent_fivegram)

[('    ', 1226640), ('$$    ', 28565), ('of ordinary skill in the', 16789), ('ordinary skill in the art', 13176), ('a person of ordinary skill', 6879), ('person of ordinary skill in', 6773), ('United States District Court for', 6308), ('States District Court for the', 6297), ('$$ United States Court of', 6235), ('judgment as a matter of', 5974), ('as a matter of law', 5940), ('one of ordinary skill in', 5870), ('the United States District Court', 5703), ('Court of Appeals for the', 5429), ('skill in the art would', 4866), ('United States Court of Appeals', 4760), ('at the time of the', 4637), ('$$ NOTES $$ [1] ', 4284), ('With him on the brief', 4272), ('one skilled in the art', 4229), ('of Appeals for the Federal', 4225), ('$$ United States District Court,', 4121), ('States Court of Appeals for', 3816), ('Appeals for the Federal Circuit', 3759), ('clear and convincing evidence that', 3627), ('genuine issue of material fact', 3526), ('by clear and convincing evidence', 3471), ('skill i

In [24]:
top_patent_bigram_list=[top_patent_bigram[i][0] for i in range(len(top_patent_bigram))]
top_patent_trigram_list=[top_patent_trigram[i][0] for i in range(len(top_patent_trigram))]
top_patent_fourgram_list=[top_patent_fourgram[i][0] for i in range(len(top_patent_fourgram))]
top_patent_fivegram_list=[top_patent_fivegram[i][0] for i in range(len(top_patent_fivegram))]


top_non_patent_bigram_list=[top_non_patent_bigram[i][0] for i in range(len(top_non_patent_bigram))]
top_non_patent_trigram_list=[top_non_patent_trigram[i][0] for i in range(len(top_non_patent_trigram))]
top_non_patent_fourgram_list=[top_non_patent_fourgram[i][0] for i in range(len(top_non_patent_fourgram))]
top_non_patent_fivegram_list=[top_non_patent_fivegram[i][0] for i in range(len(top_non_patent_fivegram))]



In [25]:
top_exclusive_patent_bigram_list=[top_patent_bigram_item for top_patent_bigram_item in top_patent_bigram_list if top_patent_bigram_item not in top_non_patent_bigram_list]
top_exclusive_patent_trigram_list=[top_patent_trigram_item for top_patent_trigram_item in top_patent_trigram_list if top_patent_trigram_item not in top_non_patent_trigram_list]
top_exclusive_patent_fourgram_list=[top_patent_fourgram_item for top_patent_fourgram_item in top_patent_fourgram_list if top_patent_fourgram_item not in top_non_patent_fourgram_list]
top_exclusive_patent_fivegram_list=[top_patent_fivegram_item for top_patent_fivegram_item in top_patent_fivegram_list if top_patent_fivegram_item not in top_non_patent_fivegram_list]


In [29]:
top_exclusive_patent_fivegram_list

['of ordinary skill in the',
 'ordinary skill in the art',
 'a person of ordinary skill',
 'person of ordinary skill in',
 'one of ordinary skill in',
 'skill in the art would',
 'one skilled in the art',
 'skill in the art to',
 'United States Patent and Trademark',
 'under the doctrine of equivalents.',
 'having ordinary skill in the',
 '    |',
 'the United States Patent and',
 'would have been obvious to',
 'person having ordinary skill in',
 'the asserted claims of the',
 'a person having ordinary skill',
 'the time the invention was',
 'level of ordinary skill in',
 'of skill in the art',
 'Court of Customs and Patent',
 'at the time the invention',
 'skilled in the art to',
 'the level of ordinary skill',
 'skilled in the art would',
 'Graham v. John Deere Co.,',
 'to one of ordinary skill',
 'than one year prior to',
 'infringement under the doctrine of',
 'in the art would have',
 'motion for summary judgment of',
 'more than one year prior',
 'in the art to which',
 '   | ',


In [39]:
with open('/home/ec2-user/SageMaker/for_Brandon/top_exclusive_patent_bigram_list.pickle', 'wb') as handle:
    pickle.dump(top_exclusive_patent_bigram_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('/home/ec2-user/SageMaker/for_Brandon/top_exclusive_patent_trigram_list.pickle', 'wb') as handle:
    pickle.dump(top_exclusive_patent_trigram_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('/home/ec2-user/SageMaker/for_Brandon/top_exclusive_patent_fourgram_list.pickle', 'wb') as handle:
    pickle.dump(top_exclusive_patent_fourgram_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('/home/ec2-user/SageMaker/for_Brandon/top_exclusive_patent_fivegram_list.pickle', 'wb') as handle:
    pickle.dump(top_exclusive_patent_fivegram_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('/home/ec2-user/SageMaker/for_Brandon/patent_files.pickle', 'wb') as handle:
    pickle.dump(patent_files, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [2]:
with open('/home/ec2-user/SageMaker/for_Brandon/top_exclusive_patent_bigram_list.pickle', 'rb') as handle:
    top_exclusive_patent_bigram_list = pickle.load(handle)

with open('/home/ec2-user/SageMaker/for_Brandon/top_exclusive_patent_trigram_list.pickle', 'rb') as handle:
    top_exclusive_patent_trigram_list = pickle.load(handle)
    
with open('/home/ec2-user/SageMaker/for_Brandon/top_exclusive_patent_fourgram_list.pickle', 'rb') as handle:
    top_exclusive_patent_fourgram_list = pickle.load(handle)
    
with open('/home/ec2-user/SageMaker/for_Brandon/top_exclusive_patent_fivegram_list.pickle', 'rb') as handle:
    top_exclusive_patent_fivegram_list = pickle.load(handle)
    
with open('/home/ec2-user/SageMaker/for_Brandon/patent_files.pickle', 'rb') as handle:
    patent_files = pickle.load(handle)
    

In [6]:
print((top_exclusive_patent_fivegram_list))

['of ordinary skill in the', 'ordinary skill in the art', 'a person of ordinary skill', 'person of ordinary skill in', 'one of ordinary skill in', 'skill in the art would', 'one skilled in the art', 'skill in the art to', 'United States Patent and Trademark', 'under the doctrine of equivalents.', 'having ordinary skill in the', '    |', 'the United States Patent and', 'would have been obvious to', 'person having ordinary skill in', 'the asserted claims of the', 'a person having ordinary skill', 'the time the invention was', 'level of ordinary skill in', 'of skill in the art', 'Court of Customs and Patent', 'at the time the invention', 'skilled in the art to', 'the level of ordinary skill', 'skilled in the art would', 'Graham v. John Deere Co.,', 'to one of ordinary skill', 'than one year prior to', 'infringement under the doctrine of', 'in the art would have', 'motion for summary judgment of', 'more than one year prior', 'in the art to which', '   | ', 'person skilled in the art', 'und

In [5]:
alice_files=[]
for i in range(len(patent_files)):
    with open(patent_files[i], 'r') as f:
        read_f = f.read().lower()
        if "alice" in read_f:
            alice_files.append(patent_files[i])

In [6]:
print(len(alice_files))

432


In [3]:
exclusive_patent_ngrams=[]
exclusive_patent_ngrams.extend(top_exclusive_patent_fivegram_list)
exclusive_patent_ngrams.extend(top_exclusive_patent_fourgram_list)
exclusive_patent_ngrams.extend(top_exclusive_patent_trigram_list)
exclusive_patent_ngrams.extend(top_exclusive_patent_bigram_list)


In [4]:
print(len(exclusive_patent_ngrams))

3786


In [18]:
for exclusive_patent_ngram in exclusive_patent_ngrams:
    print(exclusive_patent_ngram)

of ordinary skill in the
ordinary skill in the art
a person of ordinary skill
person of ordinary skill in
one of ordinary skill in
skill in the art would
one skilled in the art
skill in the art to
United States Patent and Trademark
under the doctrine of equivalents.
having ordinary skill in the
    |
the United States Patent and
would have been obvious to
person having ordinary skill in
the asserted claims of the
a person having ordinary skill
the time the invention was
level of ordinary skill in
of skill in the art
Court of Customs and Patent
at the time the invention
skilled in the art to
the level of ordinary skill
skilled in the art would
Graham v. John Deere Co.,
to one of ordinary skill
than one year prior to
infringement under the doctrine of
in the art would have
motion for summary judgment of
more than one year prior
in the art to which
   | 
person skilled in the art
under the doctrine of equivalents
TABLE OR OTHER DATA THAT
OR OTHER DATA THAT IS
OTHER DATA THAT IS NOT
to a p

used in a
the Federal Circuit,
claimed by the
the patent
prior art
35 U.S.C.
a patent
the invention
the art
the Patent
the specification
skill in
patent is
ordinary skill
Patent No.
infringement of
Federal Circuit
under 35
U.S. Patent
claim 1
Patent Office
patent and
the PTO
the patentee
patent in
patent was
skilled in
the patented
disclosed in
of patent
patent infringement
the patents
claim construction
of infringement
patent. $$
the asserted
prosecution history
patent application
the examiner
means for
in claim
prior art.
written description
Patent and
claimed invention
Claim 1
invention was
the inventor
plurality of
States Patent
the patent.
prior art,
in suit
invention is
art to
inequitable conduct
claimed in
art would
patent claims
INC. v.
been obvious
the patent,
and Trademark
of equivalents
asserted claims
The patent
a method
patent to
differences between
obvious to
patents in
that claim
for infringement
for patent
claim language
$$ Claim
infringe the
surface of
a plurality
argu

In [19]:
with open('/home/ec2-user/SageMaker/for_Brandon/exclusive_patent_ngrams.pickle', 'wb') as handle:
    pickle.dump(exclusive_patent_ngrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
no_overlap_exclusive_patent_ngrams=[]
for i in range(len(exclusive_patent_ngrams)):
    found=0
    for j in range(len(exclusive_patent_ngrams)):
        if exclusive_patent_ngrams[i] in exclusive_patent_ngrams[j] and exclusive_patent_ngrams[i] != exclusive_patent_ngrams[j]: 
            found=1
            break
    if not found:
        no_overlap_exclusive_patent_ngrams.append(exclusive_patent_ngrams[i])

In [22]:
with open('/home/ec2-user/SageMaker/for_Brandon/no_overlap_exclusive_patent_ngrams.pickle', 'wb') as handle:
    pickle.dump(no_overlap_exclusive_patent_ngrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
print(len(no_overlap_exclusive_patent_ngrams))

2065


In [23]:
for no_overlap_exclusive_patent_ngram in no_overlap_exclusive_patent_ngrams:
    print(no_overlap_exclusive_patent_ngram)

of ordinary skill in the
a person of ordinary skill
person of ordinary skill in
one of ordinary skill in
skill in the art would
skill in the art to
United States Patent and Trademark
under the doctrine of equivalents.
having ordinary skill in the
    |
the United States Patent and
would have been obvious to
person having ordinary skill in
the asserted claims of the
a person having ordinary skill
the time the invention was
level of ordinary skill in
Court of Customs and Patent
at the time the invention
skilled in the art to
the level of ordinary skill
skilled in the art would
Graham v. John Deere Co.,
to one of ordinary skill
than one year prior to
infringement under the doctrine of
in the art would have
motion for summary judgment of
more than one year prior
in the art to which
person skilled in the art
TABLE OR OTHER DATA THAT
OR OTHER DATA THAT IS
OTHER DATA THAT IS NOT
to a person having ordinary
the prior art and the
scope and content of the
and content of the prior
one year prior 

In [20]:
edited_ngrqms=["doctrine of equivalents",
               "Each party shall bear",
              ]

In [21]:
with open('/home/ec2-user/SageMaker/for_Brandon/edited_ngrqms.pickle', 'wb') as handle:
    pickle.dump(edited_ngrqms, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
print(len(no_overlap_exclusive_patent_ngrams))

2065


In [15]:
no_overlap_exclusive_patent_ngrams

['of ordinary skill in the',
 'a person of ordinary skill',
 'person of ordinary skill in',
 'one of ordinary skill in',
 'skill in the art would',
 'skill in the art to',
 'United States Patent and Trademark',
 'under the doctrine of equivalents.',
 'having ordinary skill in the',
 '    |',
 'the United States Patent and',
 'would have been obvious to',
 'person having ordinary skill in',
 'the asserted claims of the',
 'a person having ordinary skill',
 'the time the invention was',
 'level of ordinary skill in',
 'Court of Customs and Patent',
 'at the time the invention',
 'skilled in the art to',
 'the level of ordinary skill',
 'skilled in the art would',
 'Graham v. John Deere Co.,',
 'to one of ordinary skill',
 'than one year prior to',
 'infringement under the doctrine of',
 'in the art would have',
 'motion for summary judgment of',
 'more than one year prior',
 'in the art to which',
 'person skilled in the art',
 'TABLE OR OTHER DATA THAT',
 'OR OTHER DATA THAT IS',
 'OTHE

In [44]:
cleaned_exclusive_patent_ngrams=[exclusive_patent_ngram for exclusive_patent_ngram in exclusive_patent_ngrams if "35" not in exclusive_patent_ngram ]


In [45]:
print(len(cleaned_exclusive_patent_ngrams))

3665


In [42]:
with open('/home/ec2-user/SageMaker/for_Brandon/cleaned_exclusive_patent_ngrams.pickle', 'wb') as handle:
    pickle.dump(cleaned_exclusive_patent_ngrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
with open('/home/ec2-user/SageMaker/for_Brandon/cleaned_exclusive_patent_ngrams.pickle', 'rb') as handle:
    cleaned_exclusive_patent_ngrams = pickle.load(handle)

In [18]:
discarded_features=[[exclusive_patent_ngram for exclusive_patent_ngram in exclusive_patent_ngrams if exclusive_patent_ngram not in cleaned_exclusive_patent_ngrams ]]

In [19]:
print(discarded_features)

[['pursuant to 35 U.S.C. §', '35 U.S.C. § 112, ¶', 'invalid under 35 U.S.C. §', 'under 35 U.S.C. § 103', 'within the meaning of 35', 'the meaning of 35 U.S.C.', 'under 35 U.S.C. § 103.', 'under 35 U.S.C. § 112,', 'meaning of 35 U.S.C. §', '35 U.S.C. § 112 ¶', 'requirement of 35 U.S.C. §', 'of 35 U.S.C. § 112,', 'obvious under 35 U.S.C. §', 'requirements of 35 U.S.C. §', '35 U.S.C. § 103. $$', 'fees under 35 U.S.C. §', 'infringement under 35 U.S.C. §', 'under 35 U.S.C. § 112', 'under 35 U.S.C. § 102(b)', 'is invalid under 35 U.S.C.', 'of 35 U.S.C. § 112', 'obviousness under 35 U.S.C. §', 'under 35 U.S.C. § 102', 'the requirements of 35 U.S.C.', 'patent under 35 U.S.C. §', 'under 35 U.S.C. § 285', 'prior art under 35 U.S.C.', 'of 35 U.S.C. § 112.', 'under 35 U.S.C. § 285.', 'under 35 U.S.C. § 102(b).', 'in violation of 35 U.S.C.', 'violation of 35 U.S.C. §', 'attorney fees under 35 U.S.C.', 'are invalid under 35 U.S.C.', 'required by 35 U.S.C. §', 'under 35 U.S.C. § 103,', '$$ Under 35 U

In [17]:
def feature_extractor(text):
    features=[1 if cleaned_exclusive_patent_ngram in text.lower() else 0 for cleaned_exclusive_patent_ngram in cleaned_exclusive_patent_ngrams]
    return(features)

In [18]:
text="The United States Patent and Trademark Office this is a united states patent and trademark office test to see how patenting and the patent thing works here the obviousness"
extracted_features=feature_extractor(text)

In [20]:
test_extracted_features=[cleaned_exclusive_patent_ngrams[i] for i in range(len(extracted_features)) if extracted_features[i]==1 ]
print(test_extracted_features)

['and the patent', 'the patent', 'patent and', 'the obviousness']


In [48]:
features_patent_files = np.zeros(shape=(len(patent_files),len(cleaned_exclusive_patent_ngrams)))

In [54]:
print(len(patent_files))

13055


In [55]:
#here we extract a feature vector for each patent file
features = []
for i in range(len(patent_files)):
    if i%100==0:
        print(i/len(patent_files))
    with open(patent_files[i], 'r') as f:
        read_f = f.read().lower()
        features.append(feature_extractor(read_f))
                    
        

0.0
0.0076599004212945234
0.015319800842589047
0.02297970126388357
0.030639601685178094
0.038299502106472615
0.04595940252776714
0.05361930294906166
0.06127920337035619
0.0689391037916507
0.07659900421294523
0.08425890463423975
0.09191880505553428
0.0995787054768288
0.10723860589812333
0.11489850631941785
0.12255840674071238
0.13021830716200689
0.1378782075833014
0.14553810800459593
0.15319800842589046
0.16085790884718498
0.1685178092684795
0.17617770968977403
0.18383761011106856
0.19149751053236308
0.1991574109536576
0.20681731137495213
0.21447721179624665
0.22213711221754118
0.2297970126388357
0.23745691306013023
0.24511681348142475
0.2527767139027193
0.26043661432401377
0.2680965147453083
0.2757564151666028
0.28341631558789737
0.29107621600919187
0.2987361164304864
0.3063960168517809
0.31405591727307547
0.32171581769436997
0.3293757181156645
0.337035618536959
0.34469551895825357
0.35235541937954806
0.3600153198008426
0.3676752202221371
0.3753351206434316
0.38299502106472616
0.390654

In [59]:
features_patent_files=np.array(features)

In [60]:
print(features_patent_files.shape)

(13055, 3665)


In [61]:
with open('/home/ec2-user/SageMaker/for_Brandon/features_patent_files.pickle', 'wb') as handle:
    pickle.dump(features_patent_files, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [62]:
#here we extract a feature vector for each non_patent file
non_patent_features = []
for i in range(len(non_patent_files)):
    if i%300==0:
        print(i/len(non_patent_files))
    with open(non_patent_files[i], 'r') as f:
        read_f = f.read().lower()
        non_patent_features.append(feature_extractor(read_f))

0.0
0.023741690408357077
0.04748338081671415
0.07122507122507123
0.0949667616334283
0.11870845204178537
0.14245014245014245
0.16619183285849953
0.1899335232668566
0.21367521367521367
0.23741690408357075
0.2611585944919278
0.2849002849002849
0.30864197530864196
0.33238366571699907
0.3561253561253561
0.3798670465337132
0.4036087369420703
0.42735042735042733
0.45109211775878444
0.4748338081671415
0.4985754985754986
0.5223171889838556
0.5460588793922128
0.5698005698005698
0.5935422602089269
0.6172839506172839
0.6410256410256411
0.6647673314339981
0.6885090218423552
0.7122507122507122
0.7359924026590693
0.7597340930674265
0.7834757834757835
0.8072174738841406
0.8309591642924976
0.8547008547008547
0.8784425451092118
0.9021842355175689
0.9259259259259259
0.949667616334283
0.97340930674264
0.9971509971509972


In [63]:
features_non_patent_files=np.array(non_patent_features)

In [64]:
with open('/home/ec2-user/SageMaker/for_Brandon/features_non_patent_files.pickle', 'wb') as handle:
    pickle.dump(features_non_patent_files, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
patent_classifier = tf.keras.models.Sequential()
patent_classifier.add(tf.keras.Input(shape=(len(cleaned_exclusive_patent_ngrams),)))
patent_classifier.add(tf.keras.layers.Dense(1, activation='softmax'))

In [28]:
patent_classifier.compile(optimizer="Adam", loss="binary_crossentropy")
patent_classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 3666      
Total params: 3,666
Trainable params: 3,666
Non-trainable params: 0
_________________________________________________________________
